In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, output_guardrail, GuardrailFunctionOutput
from typing import Dict, List
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel, Field

import ollama

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
ollama_api_key = "http://localhost:11434/api/chat"

In [4]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:4]}")
else:
    print("Anthropic API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Anthropic API Key exists and begins sk-a


# Agents

In [5]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response. \
Sometimes you make some bad jokes about the company or the people."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

instructions4 = "You are an unfriendly and busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You don't like your job and think you are not challenged enough in your position. You think you deserved a promotion \
long time ago. You write annoyed emails so everyone can see that you hate your job. "

In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
ANTHROPIC_BASE_URL = "https://api.anthropic.com/v1/"
OOLAMA_BASE_URL = 'http://localhost:11434/v1'

In [7]:
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
anthropic_client = AsyncOpenAI(base_url=ANTHROPIC_BASE_URL, api_key=anthropic_api_key)
llama_client = AsyncOpenAI(base_url=OOLAMA_BASE_URL, api_key=ollama_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
anthropic_model = OpenAIChatCompletionsModel(model="claude-3-7-sonnet-latest", openai_client=anthropic_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=llama_client)

In [8]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model) # if you pass in text, it assumes you talkin to openai directly
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Anthropic Sales Agent",instructions=instructions3,model=anthropic_model)
sales_agent4 = Agent(name="Llama Sales Agent", instructions=instructions4, model=llama3_3_model)

In [9]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)
tool4 = sales_agent4.as_tool(tool_name="sales_agent4", tool_description=description)

In [10]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("adriana-salcedo@outlook.com")
    to_email = To("adri.salcedo@hotmail.de")  
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [11]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

preheader_instructions = "You write the inbox preview line (preheader) that complements the subject. \
Return only plain text. Use 50 chars max, use the same language as the body, no emojis and no line breaks. \
Create curiosity with the preheader."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

preheader_writer = Agent(name="Email preheader writer", instructions=preheader_instructions, model="gpt-4o-mini")
preheader_tool = preheader_writer.as_tool(tool_name="preheader_writer", tool_description="Create a preheader for the email")

In [12]:
email_tools = [subject_tool, html_tool, preheader_tool, send_html_email]

instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the preheader_writer tool to create a preheader that fit the email, then you use the  subject_writer tool \
to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [13]:
tools = [tool1, tool2, tool3, tool4]
handoffs = [emailer_agent]

class EmailOut(BaseModel):
    preheader: str
    subject: str
    body_html: str

# Guardrails

## Input Guardrails

In [14]:
class InputCheck(BaseModel):
    ok: bool
    reasoning: str
    contains_pi: bool  # contains any personal information
    type_of_pi: list[str] = [] # what kind of personal information 
    contains_hate: bool 
    contains_rudeness: bool 
    cleaned_body: str
    polite_rewrite: str = ''

guard_input_instructions = 'You are an input moderation agent for cold-email generation. \
1. Detect and report if the message contains: \
- Personal Informations (PI): emails, phone numbers, postal addresses, company internal IDs. \
- Hate speech like insults or demanding statements targeting the receiver or the company. \
- Rudeness like impoliteness, aggressive or insulting tone that is not hate speech. \
2. Rewrite the message if rudeness or aggressive behavior is detected, provide a polite rewrite but preserving the intent. \
Return only JSON matching the Input schema.'


guardrail_input_agent = Agent( 
    name="Input Check",
    instructions=guard_input_instructions,
    output_type=InputCheck,
    model="gpt-4o-mini"
)

In [15]:
@input_guardrail
async def guardrail_input(ctx, agent, message):
    result = await Runner.run(guardrail_input_agent, message, context=ctx.context)
    output = result.final_output
    trip = output.contains_hate or output.contains_pi or (not output.ok)
    body_to_use = (output.polite_rewrite or output.cleaned_body or '').strip()
    return GuardrailFunctionOutput(
        output_info={'cleaned_body': body_to_use,
                     'reasoning': output.reasoning,
        }, 
        tripwire_triggered=trip
        )

## Output Guardrails

In [16]:
class OutputCheck(BaseModel):
    ok: bool
    reasoning: str
    fixed_subject: str
    fixed_preheader: str
    fixed_body_html: str


guard_output_instructions = 'You are an output fixer for cold-email generation. \
Input fields are: preheader, subject and body_html. \
Validate and fix if necessary: \
- subject should not be longer than 60 char; single line \
- preheader not longer than 50 chars; single line; no emojis; must complement the subject and create curiosity \
- ensure that the body is converted in html \
Return only OutputCheck JSON with: \
{ok, reasoning, fixed_subject, fixed_preheader, fixed_body_htm}.'


guardrail_output_agent = Agent(
    name="Output Check",
    instructions=guard_output_instructions,
    output_type=OutputCheck,
    model='gpt-4o-mini'
)

In [17]:
@output_guardrail
async def guardrail_output(ctx, agent, output):
    payload = {
        "subject":   getattr(output, "subject", None)   or (output.get("subject")   if isinstance(output, dict) else ""),
        "preheader": getattr(output, "preheader", None) or (output.get("preheader") if isinstance(output, dict) else ""),
        "body_html": getattr(output, "body_html", None) or (output.get("body_html") if isinstance(output, dict) else ""), 
    }
    result = await Runner.run(guardrail_output_agent, output, context=ctx.context)
    output = result.final_output
    return GuardrailFunctionOutput(
        output_info={'fixed': {
                    'subject': output.fixed_subject,
                    'preheader': output.fixed_preheader,
                    'body_html': output.fixed_bofy_html,
        }, 'reasoning': output.reasoning},
        tripwire_triggered = not output.ok
                
    )

# Sales Agent

In [18]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
But don't use the tools more than 5 times. 
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    output_type=EmailOut,
    input_guardrails=[guardrail_input],
    output_guardrails=[guardrail_output],
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice (ID: 7485)"

with trace("Protected Automated SDR"):
    result = await Runner.run(sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire